In [2]:
import datasets

In [3]:
data = datasets.load_dataset("stingning/ultrachat")

Using the latest cached version of the dataset since stingning/ultrachat couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\AI\.cache\huggingface\datasets\stingning___ultrachat\default\0.0.0\f220fe796ce3ed62fbe1681b45ce6cbc9c6cabe0 (last modified on Thu Jul 11 16:41:47 2024).


Loading dataset shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [4]:
def filter_by_turns(example, turns=4, max_length=3500):
    if len(example['data']) < turns * 2:
        return False
    else:
        msg_len = sum(len(msg) for msg in example['data'])
        if msg_len < max_length and msg_len > 500:
            return True
        return False
filtered_turns_dataset = data.filter(filter_by_turns, fn_kwargs={"turns": 4, "max_length":3500})

In [5]:
filtered_turns_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'data'],
        num_rows: 58183
    })
})

In [6]:
len(filtered_turns_dataset["train"][0]['data'])

8

In [7]:
system_msg = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n"""

In [8]:
def create_conversation(sample):
    conv = [{"role": "system", "content": system_msg}]
    for idx, msg in enumerate(sample['data']):
        if idx % 2 == 0:
            conv.append({"role": "user", "content": msg})
        else:
            conv.append({"role": "assistant", "content": msg})
    return {
        "messages": conv
    }

In [9]:
new_data = filtered_turns_dataset.map(create_conversation,  remove_columns=filtered_turns_dataset['train'].features)

In [10]:
small_data= new_data['train'].train_test_split(test_size=0.05, shuffle=True)

In [12]:
small_data

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 55273
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 2910
    })
})

In [11]:
small_data.save_to_disk("convs_data")

Saving the dataset (0/1 shards):   0%|          | 0/55273 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2910 [00:00<?, ? examples/s]